<a href="https://colab.research.google.com/github/rudevico/Gachon-AISTUDY/blob/main/5_K_NN_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

앞선 예제 **2 ~ 4**에서는 **K Nearest Neighbors; K-NN; K-최근접 이웃** Algorithm을 사용해서 features(length, weight)에 따라서 data instance(생선)를 classify(bream or smelt)했다.  
즉 앞에서 사용한 **K-NN**은 엄밀히 말하자면 **K-NN Classification**이다.  
  
이번 예제에서는 **K-NN Regression**을 사용하여 주어진 문제를 해결하겠다.  
* 주어진 문제는 다음과 같다.  
  - 농어(perch)를 무게 단위로 가격을 책정하여 판매하고 있는 상황이다.  
  - 기존에 있던 농어에 대해서 길이, 높이, 두께, 무게를 측정하여 엑셀로 관리중이다.    
  - 그런데 기록자의 실수로 인해서 어떤 농어인지는 모르겠지만 다른 데이터는 모두 정상인데 무게를 실제보다 낮게 기록한 농어가 몇 개 존재한다.
  - 엑셀에 기록된 무게만 믿고 소비자에게 농어를 판매했는데, 어떤 소비자가 "이 가게는 무게를 속여서 판다."라고 리뷰를 작성했고, 이로 인해서 업장의 신뢰도가 크게 하락하는 문제가 발생했다.  
  - 현재 남아있는 농어 중 일부도 무게가 실제보다 낮게 기록되어 있는 상황인데, 저울이 고장나서 무게를 다시 잴 방법이 없다.  
  - 우리는 이 문제를 ML을 활용하여 해결하려고 한다.  
  - 다른 데이터(길이, 높이, 두께)는 모두 정확하고,
  전체 농어 중 56개는 무게 또한 정확하게 기록되어 있다.
* 우리는 주어진 문제를 다음과 같이 해결할 수 있다.  
  - **features**(x1=길이, x2=높이, x3=두께)와 **label**(Y=무게) 간의 상관관계를 파악한다.
  - 무게(label)를 잘못 기록한 농어의 경우에도 길이, 높이, 두께(features)는 정확하게 기록되어 있으므로 이로부터 무게를 예측할 수 있다.
  - 위 과정을 ***Regression**이라고 한다.  

> \*Regression analysis in statistics, a statistical technique for \*\***estimating the relationships** among variables.  
  >> \*\*estimating the relationships between a dependent variable(calld a 'label' in ML) and one or more independent variables(called a 'features' in ML).

[Download sample dataset: perch_data](http://bit.ly/perch_data)
  

* * *
위에서 features는 길이, 높이, 두께라고 언급했지만 일단은 제대로 된 길이 값만 있다면 무게를 예측할 수 있다고 가정하고 다음 과정을 진행하자.  
따라서 input = length, target = weight

In [ ]:
import numpy as np

In [ ]:
perch_length = np.array([8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0, 21.0,
       21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5, 22.5, 22.7,
       23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5, 27.3, 27.5, 27.5,
       27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0, 36.5, 36.0, 37.0, 37.0,
       39.0, 39.0, 39.0, 40.0, 40.0, 40.0, 40.0, 42.0, 43.0, 43.0, 43.5,
       44.0])
perch_weight = np.array([5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0,
       115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
       150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
       218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
       556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
       850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
       1000.0])
print(perch_length.shape)

(56,)


In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(perch_length, perch_weight, random_state=42)
print(train_input.shape, "\n", train_input)

(42,) 
 [19.6 22.  18.7 17.4 36.  25.  40.  39.  43.  22.  20.  22.  24.  27.5
 43.  40.  24.  21.  27.5 40.  32.8 26.5 36.5 13.7 22.7 15.  37.  35.
 28.7 23.5 39.  21.  23.  22.  44.  22.5 19.  37.  22.  25.6 42.  34.5]


`train_test_split()`을 통해서 split하는 데에는 문제가 없지만, 이후에 fitting 과정에서는 train set과 test set 모두 2-dim array여야 한다.  
따라서 다음 cell과 같이 2-dim array가 아니라 1-dim array로 fitting을 시도하면 ValueError가 raise된다.

In [ ]:
""" 해당 cell은 1-dim array로 fitting하려고 했을 때 ValueError가 raise되는 것을 보여주기 위한 cell입니다. """
from sklearn.neighbors import KNeighborsRegressor

knr = KNeighborsRegressor()
knr.fit(train_input, train_target)

ValueError: Expected 2D array, got 1D array instead:
array=[19.6 22.  18.7 17.4 36.  25.  40.  39.  43.  22.  20.  22.  24.  27.5
 43.  40.  24.  21.  27.5 40.  32.8 26.5 36.5 13.7 22.7 15.  37.  35.
 28.7 23.5 39.  21.  23.  22.  44.  22.5 19.  37.  22.  25.6 42.  34.5].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

> 이때 numpy array의 `reshape()` method를 사용하여 array의 shape을 변경할 수 있다.

In [ ]:
""" reshape() 예제 """
test_arr = np.array([1, 2, 3, 4])
print(test_arr.shape)
print(test_arr)

test_arr = test_arr.reshape(4, 1)
print(test_arr.shape)
print(test_arr)

(4,)
[1 2 3 4]
(4, 1)
[[1]
 [2]
 [3]
 [4]]


> If you want convert `(l, )` shape array to `(m, n)` shape array using `reshape()` method, the value of `l` is must be equal to `m*n`.

In [ ]:
train_input = train_input.reshape(-1, 1)
test_input = test_input.reshape(-1, 1)
print(train_input.shape)

(42, 1)


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knr = KNeighborsRegressor()
knr.fit(train_input, train_target)

KNeighborsRegressor()

In [ ]:
print("R^2=", knr.score(test_input, test_target))

R^2= 0.992809406101064


In [ ]:
from sklearn.metrics import mean_absolute_error

test_prediction = knr.predict(test_input)

mae = mean_absolute_error(test_target, test_prediction)
print(mae)
print("즉, 예측값은 평균적으로 실제 결과값보다 19.16만큼 차이가 난다.")

19.157142857142862
즉, 예측값은 평균적을 실제 결과값보다 19.16만큼 차이가 난다.


앞선 Clssification 문제에서는 train set으로 fitting한 Model의 train set에 대한 score는 1.0이었다.  
그런데 다음 cell을 보면 train set에 대한 score가 0.96으로, 오히려 test set에 대한 score보다 낮은 것을 확인할 수 있다.

In [ ]:
print(knr.score(train_input, train_target))

0.9698823289099254


이와 같은 현상을 ***Underfitting**이라고 한다.  
> *Underfitting occurs when a model is too simple, which can be aresult of a model eeding more training time, more input features, or less regularization.  

즉, underfitting을 해결하기 위해서는 model을 simple하지 않게 조정하면 된다.  
model을 simple하지 않게 만드는 방법에는 여러 가지가 있지만,  
K-NN Algorithm에서는 k값을 줄여서 목적을 달성할 수 있다.

In [ ]:
knr.n_neighbors = 3 # class object를 새로 만들지 않고, n_neighbors 속성값을 바꿔주면 된다.

knr.fit(train_input, train_target) # 당연하게도 model을 다시 fitting해야 한다..
print(knr.score(train_input,train_target))
print(knr.score(test_input, test_target))

0.9804899950518966
0.9746459963987609


이제 train set의 score가 test set의 score보다 높고, 그 절대적인 값 자체도 너무 작지 않으므로 underfitting되지 않았다고 판단할 수 있겠다.  
또한 train set과 score set의 차이가 너무 크지도 않으므로 overfitting되지도 않았다고 판단할 수 있겠다.